In [ ]:
# Standard library imports
import json
import pickle
from pathlib import Path
from typing import List, Dict, Any, Set
from pprint import pprint
import warnings

# Data processing
import pandas as pd
import numpy as np

# Set options
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'matplotlib'

## Configure DSPy with Azure OpenAI

Set up DSPy to use Azure OpenAI models for the dspy baseline implementation.

### Configuration:
- **Main Model**: GPT-4o for view selection and reasoning

### Important:
Make sure your environment variables are set in .env file:
- `AZURE_API_BASE`
- `AZURE_OPENAI_API_KEY`


In [1]:
import sys
print(sys.executable)
print(sys.version)

c:\Work\Portfolio\prompt-optimization-lab\.venv\Scripts\python.exe
3.12.11 (main, Jul 23 2025, 00:32:20) [MSC v.1944 64 bit (AMD64)]


In [2]:
import importlib.util
print(importlib.util.find_spec("typing_extensions"))

ModuleSpec(name='typing_extensions', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000001D6383702C0>, origin='c:\\Work\\Portfolio\\prompt-optimization-lab\\.venv\\Lib\\site-packages\\typing_extensions.py')


In [ ]:
# DSPy framework
import dspy
# Configure DSPy with your preferred LLM
# Uncomment and configure one of these options:
lm = dspy.LM("azure/gpt-4o")
dspy.configure(lm=lm)

# Option 1: OpenAI
# dspy.settings.configure(lm=dspy.OpenAI(model="gpt-4", api_key="your-api-key"))

# Option 2: Anthropic Claude
# dspy.settings.configure(lm=dspy.Claude(model="claude-3-sonnet-20240229", api_key="your-api-key"))

# Option 3: Local model (Ollama)
# dspy.settings.configure(lm=dspy.OllamaLocal(model="llama3"))
print(f"📦 DSPy version: {dspy.__version__}")

📦 DSPy version: 3.0.3


NameError: name 'pd' is not defined

In [13]:
DATA_DIR = Path("../data")
OUTPUT_DIR = Path("../data/baseline")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
sql_df = pd.read_csv("../data/golden_dataset_v1.csv")
sql_df[['question', 'selected_views_actual', 'expected_views']]

,question,selected_views_actual,expected_views
0,What is MICs current exposure in the GCC in Pr...,NaN,PLATFORM_BY_COUNTRY_VW
1,What is MICs current exposure in the USA in PE...,NaN,INVESTMENT_KPI_VW
2,What is MICs current exposure in the USA in PE...,NaN,PLATFORM_BY_COUNTRY_VW
3,What is MICs current exposure in the USA in PE?,"MIC_BY_ASSET_CLASS_VW, MIC_BY_COUNTRY_VW","PLATFORM_BY_COUNTRY_VW, INVESTMENT_KPI_VW"
4,What is MICs current exposure in the GCC in Pr...,"MIC_BY_ASSET_CLASS_VW, MIC_BY_REGION_VW","PLATFORM_BY_COUNTRY_VW, INVESTMENT_KPI_VW"
5,"In FinTech, report the current exposure for Eu...","MIC_BY_REGION_VW, MIC_BY_SECTOR_VW",<NO_VIEWS>
6,"In Pharmaceuticals, report the current exposur...","BUSINESS_UNIT_BY_SECTOR_VW, PLATFORM_BY_SECTOR_VW",INVESTMENT_KPI_VW
7,Report the exposures for Real Assets in the UAE.,"MIC_BY_ASSET_CLASS_VW, MIC_BY_COUNTRY_VW",PLATFORM_BY_REGION_VW
8,Country: Report Group exposure by country.,MIC_BY_COUNTRY_VW,MIC_BY_COUNTRY_VW
9,Region: Report Group exposure by region.,MIC_BY_REGION_VW,MIC_BY_REGION_VW


In [17]:
data =  sql_df.to_dict(orient="records")

## Define ViewSelectorSignature

Create the DSPy signature that defines inputs and outputs for the view selector.

### Signature Components:

**Inputs:**
- `question`: User's natural language query
- `candidate_views`: Available Snowflake views (with metadata)
- `conversation_history`: Previous conversation context (optional)
- `domain_knowledge`: Financial classification rules

**Outputs:**
- `reasoning`: Step-by-step analysis (Chain-of-Thought)
- `selected_views`: List of selected view entity names

### Why This Signature?
The signature guides the LLM to:
1. Analyze the question systematically
2. Consider all available views
3. Apply domain knowledge
4. Explain its reasoning
5. Return structured output

You are an expert database view selector. Given a user query, you need to identify which database views contain the relevant data.
      Call `get_entity_schema()` to get the detailed schema of the view(s) you selected, enabling subsequent agents to answer the question effectively.

      Key Instructions:
      1. Identify Relevant Views: Carefully analyze the QUESTION and the provided VIEW(s) descriptions, selectors and columns using your financial knowledge. And determine which view(s) are most likely to contain the required data to answer the QUESTION.
      2. Schema Retrieval: Call `get_entity_schema()` function to get the detailed schema of the view(s) selected.
      3. No Match Scenario: If none of the provided views appear relevant to the QUESTION, explicitly state '<NO_VIEWS>' instead of suggesting irrelevant function calls. Remember, it's totally fine to say that it's not possible. If there are no views to select, you must return '<NO_VIEWS>'.
      4. Already Attempted Scenario: If you have already provided views to the SQL Writer Agent but these do not contain the correct information, attempt to find other views that may contain this information. If there are no more views to select, you must return '<NO_VIEWS>'.
      5. CONVERSATION HISTORY: When CONVERSATION HISTORY is provided, use it to resolve ambiguous references in the current QUESTION (e.g., "them", "those companies", "similar metrics") by understanding what entities, time periods, or metrics were discussed previously.

In [8]:
class ViewSelectorSignature(dspy.Signature):
    """
    You are an expert database view selector. Given a user query, you need to identify which database views contain the relevant data.

    Key Instructions:
    1. Identify Relevant Views: Carefully analyze the QUESTION and the provided VIEW(s) descriptions, selectors and columns using your financial knowledge. Determine which view(s) are most likely to contain the required data to answer the QUESTION.
    2. No Match Scenario: If none of the provided views appear relevant to the QUESTION, explicitly return ['<NO_VIEWS>'] instead of suggesting irrelevant views. Remember, it's totally fine to say that it's not possible.
    3. Already Attempted Scenario: If views have already been provided but do not contain the correct information, attempt to find other views that may contain this information. If there are no more views to select, return ['<NO_VIEWS>'].
    4. CONVERSATION HISTORY: When CONVERSATION HISTORY is provided, use it to resolve ambiguous references in the current QUESTION
       (e.g., "them", "those companies", "similar metrics") by understanding what entities, time periods, or metrics were discussed previously.

    Analyzes a natural language question and selects the most relevant Snowflake database views needed to answer the question.
    """
    
    # Input fields
    question: str = dspy.InputField(
        desc="User's natural language database query"
    )
    
    candidate_views: list = dspy.InputField(
        desc="List of available database views with descriptions, selectors, and columns"
    )
    
    conversation_history: str = dspy.InputField(
        desc="Previous conversation context for resolving references",
        default=""
    )
    
    domain_knowledge: str = dspy.InputField(
        desc="Financial domain rules (Asset Classes, Investment Classes, Platforms, Business Units)"
    )
    
    # Output fields
    reasoning: str = dspy.OutputField(
        desc="Step-by-step analysis of why specific views were selected"
    )
    
    selected_views: list = dspy.OutputField(
        desc="List of selected view entity names, or ['<NO_VIEWS>'] if none are relevant"
    )

## Implement ViewSelectorModule

Create the baseline DSPy module using Chain-of-Thought reasoning.

### Module Features:
1. **Chain-of-Thought**: Uses `dspy.ChainOfThought` for step-by-step reasoning
2. **Domain Knowledge**: Embeds financial classification rules
3. **Candidate Views**: Uses Snowflake metadata for view selection
4. **Clean Interface**: Simple `forward()` method for predictions

### Domain Knowledge Embedded:
- **Investment Classes**: Private Equity, Real Estate, Infrastructure, Credit, Hedge Funds
- **Asset Classes**: Same as Investment Classes
- **Platforms**: Private Equity, Real Assets, ADIC, UAE Investments, Credit and Special Situations

### Important:
This is the **unoptimized baseline**. No few-shot examples, no prompt engineering, just the base DSPy ChainOfThought module.

In [9]:
class ViewSelectorModule(dspy.Module):
    """
    Baseline database view selector using DSPy Chain-of-Thought reasoning.
    
    This module implements the core logic:
    1. Analyze the question and available views
    2. Apply financial/domain knowledge
    3. Handle conversation history for context
    4. Return appropriate views or <NO_VIEWS>
    """
    
    def __init__(self, candidate_views: List[Dict] = None):
        super().__init__()
        
        # Use Chain of Thought for step-by-step reasoning
        self.selector_cot = dspy.ChainOfThought(ViewSelectorSignature)
        
        # Embed domain knowledge
        self.domain_knowledge = """
CRITICAL FINANCIAL CLASSIFICATION RULES:

1. INVESTMENT CLASSES:
   - Available values: Private Equity, Real Estate, Infrastructure, Credit, Hedge Funds
   - These are broad investment categories

2. ASSET CLASSES:
   - Available values: Private Equity, Real Estate, Infrastructure, Credit, Hedge Funds
   - Note: PE refers to Private Equity (an ASSET CLASS, not just an abbreviation)

3. PLATFORMS:
   - Available platform values: Private Equity, Real Assets, ADIC, UAE Investments, 
     Credit and Special Situations
   - If a platform is mentioned but not in this list, it may not be relevant

4. CRITICAL NOTES:
   - "PE" typically refers to "Private Equity" (the asset class)
   - Country/geography questions need location-aware views
   - Exposure questions need portfolio/position views
   - Fund-level questions need fund detail views
        """
        
        # Store candidate views
        self.candidate_views = candidate_views or []
    
    def forward(self, question: str, conversation_history: str = ""):
        """
        Select the most relevant database views for the given question.
        
        Args:
            question: User's database query
            conversation_history: Previous conversation context
            
        Returns:
            dspy.Prediction with reasoning and selected_views
        """
        # Use Chain-of-Thought reasoning to select views
        cot_result = self.selector_cot(
            question=question,
            candidate_views=self.candidate_views,
            conversation_history=conversation_history,
            domain_knowledge=self.domain_knowledge
        )
        
        # Return the Prediction object directly
        return cot_result

print("✅ ViewSelectorModule implemented successfully!")
print("\n📋 Module Structure:")
print("   • Uses dspy.ChainOfThought for reasoning")
print("   • Embeds financial domain knowledge")
print("   • Accepts candidate views as input")
print("   • Returns Prediction with reasoning + selected_views")

✅ ViewSelectorModule implemented successfully!

📋 Module Structure:
   • Uses dspy.ChainOfThought for reasoning
   • Embeds financial domain knowledge
   • Accepts candidate views as input
   • Returns Prediction with reasoning + selected_views


### Initialize Baseline Module

Create an instance of the ViewSelectorModule with Snowflake views as candidates.

In [16]:
# Load nowflake views metadata
with open(DATA_DIR / "snowflake_view.json", "r") as f:
    snowflake_views = json.load(f)
# Create baseline view selector
baseline_view_selector = ViewSelectorModule(candidate_views=snowflake_views)


## Test Baseline on Sample Examples

Run the baseline module on a few examples to verify it's working correctly.

In [19]:
print("🧪 TESTING BASELINE ON SAMPLE EXAMPLES")
print("=" * 60)

# Test on first 3 training examples
sample_examples = data[:3]

for i, example in enumerate(sample_examples, 1):
    print(f"\n{'─'*60}")
    print(f"Example {i}:")
    print(f"{'─'*60}")
    print(f"Question: {example['question']}...")
    print(f"Expected: {example['expected_views']}")
    
    try:
        # Run baseline prediction
        prediction = baseline_view_selector(
            question=example['question'],
            conversation_history=example.get('conversation_history', '')
        )
        
        # Display results
        print(f"   Selected Views: {prediction.selected_views}")
        print(f"   Reasoning:")
        print(f"   {prediction.reasoning}")

        # Check if prediction matches expected
        pred_set = set(str(v).strip() for v in prediction.selected_views)
        exp_views = example['expected_views']
        if exp_views and exp_views != '<NO_VIEWS>':
            exp_set = set(v.strip() for v in exp_views.split(','))
        else:
            exp_set = set()
        
        match = pred_set == exp_set
        print(f"   Match: {'✅ Yes' if match else '❌ No'}")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")


🧪 TESTING BASELINE ON SAMPLE EXAMPLES

────────────────────────────────────────────────────────────
Example 1:
────────────────────────────────────────────────────────────
Question: What is MICs current exposure in the GCC in Private Equity platform?...
Expected: PLATFORM_BY_COUNTRY_VW
   Selected Views: ['ACTIVE_DEAL_LIST_VW', 'INVESTMENT_KPI_VW', 'PLATFORM_BY_REGION_VW']
   Reasoning:
   The question asks for MIC's current exposure in the GCC within the Private Equity platform. To address this, we need to identify views that provide information on exposure, specifically within the Private Equity platform and the GCC region. 

1. **Active Deal List View (ACTIVE_DEAL_LIST_VW)**: This view contains details about ongoing deals, including platform-specific details and geographical information. It includes columns for PLATFORM and GEOGRAPHY, which can be used to filter for Private Equity and GCC countries (e.g., Saudi Arabia, United Arab Emirates, etc.). This view is relevant for understan